In [1]:
# importing dependencies
!pip install keras_tuner
import pandas as pd
import sklearn as skl
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 24.9 MB/s eta 0:00:00
Using TensorFlow backend


# Preprocessing Data


In [6]:
# importing data and converting to pandas df.
# I have created another df here "application_df_2" because I would like to see how the model does without the alterations made to
# the 'APPLICATION_TYPE' and 'CLASSIFICATION' columns

application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df_2 = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")

In [7]:
# Dropping EIN and NAME columns
application_df = application_df.drop(columns=['EIN','NAME'])
application_df_2 = application_df_2.drop(columns=['EIN','NAME'])


In [8]:
# Finding application types with frequency less than 500
type_counts=application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace=[type_counts.index[i] for i in range(len(type_counts)) if type_counts[i]<500]

# combining into one category
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

In [9]:
# Finding classifications with frequency less than 1000 thousand
class_counts=application_df['CLASSIFICATION'].value_counts()
classifications_to_replace=class_counts.loc[lambda x : x<1000].index

# combining these into one category

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")



In [10]:
# Converting categorical data to numeric

dummies = pd.get_dummies(application_df[["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"]])

# Concatenate the dummies to original dataframe
applications_cleaned_df = pd.concat([application_df, dummies], axis=1)
applications_cleaned_df_2 = pd.concat([application_df_2, dummies], axis=1)

# drop the values
applications_cleaned_df=applications_cleaned_df.drop(["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"], axis=1)
applications_cleaned_df_2=applications_cleaned_df_2.drop(["APPLICATION_TYPE","AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","INCOME_AMT","SPECIAL_CONSIDERATIONS"], axis=1)

applications_cleaned_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Splitting data into testing and training data and standardising

X = applications_cleaned_df.drop(columns=["IS_SUCCESSFUL"])
y = applications_cleaned_df["IS_SUCCESSFUL"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# This code does the same as the previous cell but for applications_cleaned_df_2

# Splitting data into testing and training data and standardising

X_2 = applications_cleaned_df_2.drop(columns=["IS_SUCCESSFUL"])
y_2 = applications_cleaned_df_2["IS_SUCCESSFUL"]

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, random_state=42, stratify=y_2)


# Fit the StandardScaler
X_scaler_2 = StandardScaler().fit(X_train_2)

# Scale the data
X_train_scaled_2 = X_scaler_2.transform(X_train_2)
X_test_scaled_2 = X_scaler_2.transform(X_test_2)

# First attempt at optimisation

In [18]:
import math
# In this attempt I will add another layer of neurons in case the model is underfitting

# input dimension
num_features = X.shape[1]

# First layer of neurons

num_neurons_1 = num_features

# Second layer (first hidden layer)
num_neurons_2 = math.floor(num_features/2)

# Third layer (second hidden layer)
num_neurons_3 = num_neurons_2

# Fourth layer (third hidden layer)
num_neurons_4 = num_neurons_2

# initalising model

nn_1 = tf.keras.models.Sequential()

# input layer

nn_1.add(tf.keras.layers.Dense(units=num_neurons_1, activation="sigmoid", input_dim=num_features))

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=num_neurons_2, activation="sigmoid"))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=num_neurons_3, activation="sigmoid"))

# Third hidden layer
nn_1.add(tf.keras.layers.Dense(units=num_neurons_4, activation="sigmoid"))

# Output layer (single neuron)
nn_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 43)                1892      
                                                                 
 dense_5 (Dense)             (None, 21)                924       
                                                                 
 dense_6 (Dense)             (None, 21)                462       
                                                                 
 dense_7 (Dense)             (None, 21)                462       
                                                                 
 dense_8 (Dense)             (None, 1)                 22        
                                                                 
Total params: 3762 (14.70 KB)
Trainable params: 3762 (14.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
import keras as keras

nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn_1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.6123 - accuracy: 0.6795
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5738 - accuracy: 0.7311
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5720 - accuracy: 0.7310
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5681 - accuracy: 0.7320
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5619 - accuracy: 0.7316
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5586 - accuracy: 0.7322
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7322
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5554 - accuracy: 0.7326
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5543 - accuracy: 0.7332
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5530 - accuracy: 0.7326

In [20]:
model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 1s 1ms/step - loss: 0.5542 - accuracy: 0.7249
Loss: 0.5541577935218811, Accuracy: 0.7248979806900024


In [26]:
nn_1.save("second_model.h5")

# Second attempt at optimisation

In [27]:
import math
# In this attempt I will more neurons to each layer in case the model is underfitting

# input dimension
num_features = X.shape[1]

# We require at least as many neurons in the input layer as there are features, so initally we can let

num_neurons_1 = num_features

# For an initial guess we can let the number of neurons in the first hidden layer to be

num_neurons_2 = num_features

# And the number of neurons in the second hidden layer to be

num_neurons_3 = num_features

# initalising model

nn_2 = tf.keras.models.Sequential()

# initially we can just use the sigmoid for all activation functions, though
# the output layer will need a sigmoid activation function since we are doing binary classification

# input layer

nn_2.add(tf.keras.layers.Dense(units=num_neurons_1, activation="sigmoid", input_dim=num_features))

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=num_neurons_2, activation="sigmoid"))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=num_neurons_3, activation="sigmoid"))

# Output layer (single neuron)
nn_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 43)                1892      
                                                                 
 dense_14 (Dense)            (None, 43)                1892      
                                                                 
 dense_15 (Dense)            (None, 43)                1892      
                                                                 
 dense_16 (Dense)            (None, 1)                 44        
                                                                 
Total params: 5720 (22.34 KB)
Trainable params: 5720 (22.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn_2.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6054 - accuracy: 0.6909
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5718 - accuracy: 0.7259
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5684 - accuracy: 0.7258
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5636 - accuracy: 0.7280
Epoch 5/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5603 - accuracy: 0.7297
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5578 - accuracy: 0.7316
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5559 - accuracy: 0.7314
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5542 - accuracy: 0.7326
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5532 - accuracy: 0.7324
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7317

In [29]:
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 1s 1ms/step - loss: 0.5547 - accuracy: 0.7256
Loss: 0.5547152161598206, Accuracy: 0.7255976796150208


In [30]:
nn_2.save("third_model.h5")

# Third attempt at optimisation

In [31]:
# In this attempt I will use the original model used in "first_attempt.ipynb" but with
# without binning the 'APPLICATION_TYPE' and 'CLASSIFICATION' columns

import math
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# input dimension
num_features = X.shape[1]


num_neurons_1 = num_features


num_neurons_2 = math.floor(num_features/2)


num_neurons_3 = math.floor(num_neurons_2)

# initalising model

nn_3 = tf.keras.models.Sequential()

# input layer

nn_3.add(tf.keras.layers.Dense(units=num_neurons_1, activation="sigmoid", input_dim=num_features))

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=num_neurons_2, activation="sigmoid"))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=num_neurons_3, activation="sigmoid"))

# Output layer (single neuron)
nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 43)                1892      
                                                                 
 dense_18 (Dense)            (None, 21)                924       
                                                                 
 dense_19 (Dense)            (None, 21)                462       
                                                                 
 dense_20 (Dense)            (None, 1)                 22        
                                                                 
Total params: 3300 (12.89 KB)
Trainable params: 3300 (12.89 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn_3.fit(X_train_scaled_2, y_train_2, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6080 - accuracy: 0.6883
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5696 - accuracy: 0.7261
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5647 - accuracy: 0.7283
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5607 - accuracy: 0.7300
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5581 - accuracy: 0.7306
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5562 - accuracy: 0.7317
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7315
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7317
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7315
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7332

In [16]:
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled_2,y_test_2,verbose=1)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 [==============================] - 2s 5ms/step - loss: 0.5536 - accuracy: 0.7264
Loss: 0.5535574555397034, Accuracy: 0.7264139652252197


In [33]:
nn_3.save("fourth_model.h5")